In [84]:
#Raghav

In [85]:
# Load CSV
import pandas as pd
import numpy as np
import re
from sklearn.preprocessing import LabelEncoder

df = pd.read_csv('train.csv')

# Drop the unnecessary columns: 'Id', 'Name', 'Found Location', 'Outcome Time', 'Date of Birth'
df = df.drop(columns=['Id', 'Name', 'Found Location', 'Outcome Time', 'Date of Birth'], axis=1)




In [86]:
def preprocess_train_data(df):
    oneHotEncodeList = []
    label_encoders = {}

    # Time features
    df['hour'] = pd.to_datetime(df['Intake Time']).dt.hour
    df['dayofweek'] = pd.to_datetime(df['Intake Time']).dt.dayofweek
    df['month'] = pd.to_datetime(df['Intake Time']).dt.month
    df['hour_sin'] = np.sin(2 * np.pi * df['hour'] / 24)
    df['hour_cos'] = np.cos(2 * np.pi * df['hour'] / 24)
    df['dayofweek_sin'] = np.sin(2 * np.pi * df['dayofweek'] / 7)
    df['dayofweek_cos'] = np.cos(2 * np.pi * df['dayofweek'] / 7)
    df['month_sin'] = np.sin(2 * np.pi * (df['month'] - 1) / 12)
    df['month_cos'] = np.cos(2 * np.pi * (df['month'] - 1) / 12)
    df.drop(columns=['Intake Time', 'hour', 'dayofweek', 'month'], inplace=True)

    # Intake Type
    df = df[df['Intake Type'] != 'Wildlife'].copy()
    oneHotEncodeList.append('Intake Type')

    # Intake Condition
    def group_intake_condition(condition):
        if pd.isnull(condition): return 'Other'
        condition = condition.lower()
        if condition in ['med attn', 'medical', 'med urgent', 'neurologic', 'congenital', 'parvo', 'agonal']:
            return 'Medical-related'
        elif condition in ['neonatal', 'aged', 'pregnant', 'nursing']:
            return 'Life stage'
        elif condition in ['normal', 'injured', 'sick']:
            return 'Health Status'
        elif condition in ['behavior', 'feral']:
            return 'Behavioral'
        else:
            return 'Other'
    df.loc[:, 'Intake Condition'] = df['Intake Condition'].apply(group_intake_condition)
    oneHotEncodeList.append('Intake Condition')

    # Animal Type
    oneHotEncodeList.append('Animal Type')

    # Sex and Fix Status
    df.loc[:, 'Sex upon Intake'] = df['Sex upon Intake'].fillna('Unknown')
    def extract_sex_and_status(sex):
        sex = str(sex).strip().lower()
        if "neutered" in sex:
            status = "Neutered"
        elif "spayed" in sex:
            status = "Spayed"
        elif "intact" in sex:
            status = "Intact"
        else:
            status = "Unknown"
        if "male" in sex:
            gender = "Male"
        elif "female" in sex:
            gender = "Female"
        else:
            gender = "Unknown"
        return pd.Series([gender, status])
    df[['Sex', 'Fixed_Status']] = df['Sex upon Intake'].apply(extract_sex_and_status)
    df.drop('Sex upon Intake', axis=1, inplace=True)
    oneHotEncodeList.extend(['Sex', 'Fixed_Status'])

    # Age
    def convert_age_to_days(age_str):
        if pd.isnull(age_str): return np.nan
        num, unit = age_str.split()[:2]
        num = int(num)
        if 'day' in unit: return num
        elif 'week' in unit: return num * 7
        elif 'month' in unit: return num * 30
        elif 'year' in unit: return num * 365
        return np.nan
    df.loc[:, 'Age upon Intake'] = df['Age upon Intake'].apply(convert_age_to_days)
    df['Age upon Intake'] = df['Age upon Intake'].apply(lambda x: np.nan if x < 0 else x)
    df['Age upon Intake'] = df['Age upon Intake'].fillna(df['Age upon Intake'].median())
    df['Log_Age'] = np.log1p(df['Age upon Intake'])

        
    color_groups = {
        'Dark': ['Black', 'Chocolate', 'Seal'],
        'Light': ['White', 'Cream', 'Buff', 'Silver'],
        'Warm': ['Red', 'Orange', 'Flame', 'Gold', 'Apricot'],
        'Cool': ['Blue', 'Gray', 'Lilac'],
        'Neutral': ['Tan', 'Brown', 'Fawn', 'Yellow', 'Liver', 'Pink', 'Ruddy']
    }

    pattern_groups = {
        'Striped': ['Tabby', 'Tiger', 'Lynx'],
        'Blotched': ['Tortie', 'Calico', 'Torbie'],
        'Gradient': ['Smoke', 'Point', 'Sable'],
        'Mixed': ['Merle', 'Brindle', 'Tricolor'],
        'Textured': ['Tick', 'Agouti'],
        'None': []
    }


    color_to_group = {c: g for g, clist in color_groups.items() for c in clist}
    pattern_to_group = {p: g for g, plist in pattern_groups.items() for p in plist}

    # Group assignment functions
    def assign_color_group(color_str):
        if pd.isnull(color_str): return "Unknown"
        for part in re.split(r'[/ ]+', color_str):
            name = part.strip().title()
            if name in color_to_group:
                return color_to_group[name]
        return "Other"

    def assign_pattern_group(color_str):
        if pd.isnull(color_str): return "None"
        for part in re.split(r'[/ ]+', color_str):
            name = part.strip().title()
            if name in pattern_to_group:
                return pattern_to_group[name]
        return "None"

    # Apply to DataFrame
    df['Color_Group'] = df['Color'].apply(assign_color_group)
    df['Pattern_Group'] = df['Color'].apply(assign_pattern_group)

    # Drop the original 'Color' column
    df = df.drop(columns=['Color'], axis=1)

    oneHotEncodeList.append('Color_Group')
    oneHotEncodeList.append('Pattern_Group')

    # Breed
    def process_breed(breed):
        if pd.isnull(breed): return pd.Series(["Unknown", True])
        is_mix = "Mix" in breed or "/" in breed
        if "/" in breed:
            primary = breed.split("/")[0].strip()
        else:
            primary = breed.replace(" Mix", "").strip()
        return pd.Series([primary, is_mix])
    df[['Primary_Breed', 'Is_Mix']] = df['Breed'].apply(process_breed)
    df['Is_Mix'] = df['Is_Mix'].astype(int)
    df.drop(columns=['Breed'], inplace=True)

    vc = df['Primary_Breed'].value_counts()
    cumulative = vc.cumsum() / vc.sum()
    top_breeds = cumulative[cumulative <= 0.90].index.tolist()
    df['Primary_Breed'] = df['Primary_Breed'].apply(lambda x: x if x in top_breeds else 'Other')
    le_breed = LabelEncoder()
    df['Primary_Breed'] = le_breed.fit_transform(df['Primary_Breed'])
    label_encoders['Primary_Breed'] = le_breed

    for col in oneHotEncodeList:
        le = LabelEncoder()
        df[col] = le.fit_transform(df[col])
        label_encoders[col] = le
    # df = pd.get_dummies(df, columns=oneHotEncodeList, drop_first=True)

    features = df.drop(columns=['Outcome Type'])
    labels = df['Outcome Type']
    return features, labels, label_encoders, top_breeds


In [87]:
from sklearn.ensemble import AdaBoostClassifier
from sklearn.tree import DecisionTreeClassifier
from sklearn.model_selection import GridSearchCV, StratifiedKFold
from sklearn.metrics import classification_report, make_scorer, f1_score
from imblearn.over_sampling import SMOTE
from imblearn.pipeline import Pipeline
import numpy as np
from xgboost import XGBClassifier

label_encoder = LabelEncoder()
# Load your dataset preprocess_shelter_data(df, is_train=True, label_encoder=None)
X, y, label_encoders, top_breeds = preprocess_train_data(df)

y = label_encoder.fit_transform(y)

# Label encode the target variable
if y.dtype == "object" or y.dtype.name == "category":
    y = label_encoder.fit_transform(y)


class_mapping = dict(zip(label_encoder.classes_, label_encoder.transform(label_encoder.classes_)))
# {'Adoption': 0, 'Died': 1, 'Euthanasia': 2, 'Return to Owner': 3, 'Transfer': 4}

smote_strategy = {
    class_mapping['Died']: 1000,
    class_mapping['Euthanasia']: 3000
}


pipeline = Pipeline([
    ("smote", SMOTE(sampling_strategy='not majority', random_state=42)),
    ("xgb", XGBClassifier(random_state=42, use_label_encoder=False, eval_metric='mlogloss'))
])


# Define parameter grid
# param_grid = {
#     "adaboost__estimator__max_depth": [1, 2, 3],
#     "adaboost__n_estimators": [50, 100, 150],
#     "adaboost__learning_rate": [0.5, 1.0]
# }


param_grid = {
    "xgb__max_depth": [7, 8, 9, 10],
    "xgb__n_estimators": [150, 200, 250],
    "xgb__subsample": [0.7, 0.8, 0.9],
}



grid_search = GridSearchCV(pipeline, param_grid, scoring='f1_macro', cv=5, verbose=1, n_jobs=-1)

# Fit
grid_search.fit(X, y)

# Best model evaluation
print("✅ Best Parameters:", grid_search.best_params_)
print("✅ Best CV Macro F1 Score:", grid_search.best_score_)


Fitting 5 folds for each of 36 candidates, totalling 180 fits


c:\Users\Raghav\AppData\Local\Programs\Python\Python312\Lib\site-packages\xgboost\training.py:183: UserWarning: [00:12:16] WARNING: C:\actions-runner\_work\xgboost\xgboost\src\learner.cc:738: 
Parameters: { "use_label_encoder" } are not used.

  bst.update(dtrain, iteration=i, fobj=obj)


✅ Best Parameters: {'xgb__max_depth': 9, 'xgb__n_estimators': 150, 'xgb__subsample': 0.9}
✅ Best CV Macro F1 Score: 0.41035346719961885


In [88]:
# Evaluate the best model on the training set
from sklearn.metrics import classification_report, confusion_matrix
from sklearn.metrics import accuracy_score, f1_score
best_model = grid_search.best_estimator_
y_pred = best_model.predict(X)
print("✅ Training Set Classification Report:")
print(classification_report(y, y_pred))
print(confusion_matrix(y, y_pred))


✅ Training Set Classification Report:
              precision    recall  f1-score   support

           0       0.74      0.90      0.81     55044
           1       0.82      0.54      0.65      1041
           2       0.92      0.49      0.64      3449
           3       0.78      0.72      0.75     16598
           4       0.82      0.63      0.71     35024

    accuracy                           0.77    111156
   macro avg       0.81      0.66      0.71    111156
weighted avg       0.78      0.77      0.76    111156

[[49514    58    57  1942  3473]
 [  310   565     1    14   151]
 [ 1063     8  1706   146   526]
 [ 3971     1    17 11940   669]
 [11663    61    79  1318 21903]]


In [ ]:
def preprocess_test_data(df, label_encoders, top_breeds):
    import re

    df['hour'] = pd.to_datetime(df['Intake Time']).dt.hour
    df['dayofweek'] = pd.to_datetime(df['Intake Time']).dt.dayofweek
    df['month'] = pd.to_datetime(df['Intake Time']).dt.month
    df['hour_sin'] = np.sin(2 * np.pi * df['hour'] / 24)
    df['hour_cos'] = np.cos(2 * np.pi * df['hour'] / 24)
    df['dayofweek_sin'] = np.sin(2 * np.pi * df['dayofweek'] / 7)
    df['dayofweek_cos'] = np.cos(2 * np.pi * df['dayofweek'] / 7)
    df['month_sin'] = np.sin(2 * np.pi * (df['month'] - 1) / 12)
    df['month_cos'] = np.cos(2 * np.pi * (df['month'] - 1) / 12)
    df.drop(columns=['Intake Time', 'hour', 'dayofweek', 'month'], inplace=True)

    def group_intake_condition(condition):
        if pd.isnull(condition): return 'Other'
        condition = condition.lower()
        if condition in ['med attn', 'medical', 'med urgent', 'neurologic', 'congenital', 'parvo', 'agonal']:
            return 'Medical-related'
        elif condition in ['neonatal', 'aged', 'pregnant', 'nursing']:
            return 'Life stage'
        elif condition in ['normal', 'injured', 'sick']:
            return 'Health Status'
        elif condition in ['behavior', 'feral']:
            return 'Behavioral'
        else:
            return 'Other'
    df['Intake Condition'] = df['Intake Condition'].apply(group_intake_condition)

    df['Sex upon Intake'] = df['Sex upon Intake'].fillna('Unknown')
    def extract_sex_and_status(sex):
        sex = str(sex).strip().lower()
        if "neutered" in sex:
            status = "Neutered"
        elif "spayed" in sex:
            status = "Spayed"
        elif "intact" in sex:
            status = "Intact"
        else:
            status = "Unknown"
        if "male" in sex:
            gender = "Male"
        elif "female" in sex:
            gender = "Female"
        else:
            gender = "Unknown"
        return pd.Series([gender, status])
    df[['Sex', 'Fixed_Status']] = df['Sex upon Intake'].apply(extract_sex_and_status)
    df.drop('Sex upon Intake', axis=1, inplace=True)

    def convert_age_to_days(age_str):
        if pd.isnull(age_str): return np.nan
        num, unit = age_str.split()[:2]
        num = int(num)
        if 'day' in unit: return num
        elif 'week' in unit: return num * 7
        elif 'month' in unit: return num * 30
        elif 'year' in unit: return num * 365
        return np.nan
    df['Age upon Intake'] = df['Age upon Intake'].apply(convert_age_to_days)
    df['Age upon Intake'] = df['Age upon Intake'].apply(lambda x: np.nan if x < 0 else x)
    df['Age upon Intake'] = df['Age upon Intake'].fillna(df['Age upon Intake'].median())
    df['Log_Age'] = np.log1p(df['Age upon Intake'])

    def process_breed(breed):
        if pd.isnull(breed): return pd.Series(["Unknown", True])
        is_mix = "Mix" in breed or "/" in breed
        if "/" in breed:
            primary = breed.split("/")[0].strip()
        else:
            primary = breed.replace(" Mix", "").strip()
        return pd.Series([primary, is_mix])
    df[['Primary_Breed', 'Is_Mix']] = df['Breed'].apply(process_breed)
    df['Is_Mix'] = df['Is_Mix'].astype(int)
    df.drop(columns=['Breed'], inplace=True)

        
    color_groups = {
        'Dark': ['Black', 'Chocolate', 'Seal'],
        'Light': ['White', 'Cream', 'Buff', 'Silver'],
        'Warm': ['Red', 'Orange', 'Flame', 'Gold', 'Apricot'],
        'Cool': ['Blue', 'Gray', 'Lilac'],
        'Neutral': ['Tan', 'Brown', 'Fawn', 'Yellow', 'Liver', 'Pink', 'Ruddy']
    }

    pattern_groups = {
        'Striped': ['Tabby', 'Tiger', 'Lynx'],
        'Blotched': ['Tortie', 'Calico', 'Torbie'],
        'Gradient': ['Smoke', 'Point', 'Sable'],
        'Mixed': ['Merle', 'Brindle', 'Tricolor'],
        'Textured': ['Tick', 'Agouti'],
        'None': []
    }


    color_to_group = {c: g for g, clist in color_groups.items() for c in clist}
    pattern_to_group = {p: g for g, plist in pattern_groups.items() for p in plist}

    # Group assignment functions
    def assign_color_group(color_str):
        if pd.isnull(color_str): return "Unknown"
        for part in re.split(r'[/ ]+', color_str):
            name = part.strip().title()
            if name in color_to_group:
                return color_to_group[name]
        return "Other"

    def assign_pattern_group(color_str):
        if pd.isnull(color_str): return "None"
        for part in re.split(r'[/ ]+', color_str):
            name = part.strip().title()
            if name in pattern_to_group:
                return pattern_to_group[name]
        return "None"

    # Apply to DataFrame
    df['Color_Group'] = df['Color'].apply(assign_color_group)
    df['Pattern_Group'] = df['Color'].apply(assign_pattern_group)

    # Drop the original 'Color' column
    df = df.drop(columns=['Color'], axis=1)

    label_encoders = LabelEncoder()
    df['Primary_Breed'] = df['Primary_Breed'].apply(lambda x: x if x in top_breeds else 'Other')
    le_breed = label_encoders['Primary_Breed']
    df['Primary_Breed'] = df['Primary_Breed'].apply(lambda x: x if x in le_breed.classes_ else 'Other')
    if 'Other' not in le_breed.classes_:
        le_breed.classes_ = np.append(le_breed.classes_, 'Other')
    df['Primary_Breed'] = le_breed.transform(df['Primary_Breed'])

    for col in ['Intake Type', 'Intake Condition', 'Animal Type', 'Sex', 'Fixed_Status', 'Color_Group', 'Pattern_Group']:
        le = label_encoders[col]
        df[col] = df[col].apply(lambda x: x if x in le.classes_ else le.classes_[0])
        df[col] = le.transform(df[col])

    return df


In [97]:
# Try the best model on the unlabeled test set
data = pd.read_csv('test.csv')
df_test = pd.read_csv('test.csv')
df_test = preprocess_test_data(df_test,label_encoders=label_encoder, top_breeds=top_breeds)


df_test = df_test.drop(columns=['Id', 'Date of Birth', 'Found Location'], axis=1)

y_pred_test = best_model.predict(df_test)
y_pred_test = label_encoder.inverse_transform(y_pred_test)
df_test['Outcome Type'] = y_pred_test

submission = pd.DataFrame({
    'Id': data['Id'],
    'Outcome Type': y_pred_test
})
submission.to_csv('submission.csv', index=False)

print("✅ Test Set Predictions Saved to submission.csv")


C:\Users\Raghav\AppData\Local\Temp\ipykernel_20024\3944122547.py:4: UserWarning: Could not infer format, so each element will be parsed individually, falling back to `dateutil`. To ensure parsing is consistent and as-expected, please specify a format.
  df['hour'] = pd.to_datetime(df['Intake Time']).dt.hour
C:\Users\Raghav\AppData\Local\Temp\ipykernel_20024\3944122547.py:5: UserWarning: Could not infer format, so each element will be parsed individually, falling back to `dateutil`. To ensure parsing is consistent and as-expected, please specify a format.
  df['dayofweek'] = pd.to_datetime(df['Intake Time']).dt.dayofweek
C:\Users\Raghav\AppData\Local\Temp\ipykernel_20024\3944122547.py:6: UserWarning: Could not infer format, so each element will be parsed individually, falling back to `dateutil`. To ensure parsing is consistent and as-expected, please specify a format.
  df['month'] = pd.to_datetime(df['Intake Time']).dt.month


TypeError: 'LabelEncoder' object is not subscriptable